<a href="https://colab.research.google.com/github/OtolaHan/Otola_Han_KAAN/blob/main/Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2024-02-26 16:27:18--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2024-02-26 16:27:18--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.08s   

2024-02-26 16:27:19 (31.4 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/26

In [2]:

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-05.png', 'happy2-04.png', 'happy2-17.png', 'happy1-18.png', 'happy2-16.png', 'happy1-12.png', 'happy1-14.png', 'happy1-03.png', 'happy1-05.png', 'happy2-11.png']
['sad1-16.png', 'sad2-02.png', 'sad2-15.png', 'sad1-08.png', 'sad1-18.png', 'sad1-15.png', 'sad1-06.png', 'sad1-00.png', 'sad2-10.png', 'sad2-07.png']


In [5]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-05.png', 'happy2-04.png', 'happy2-17.png', 'happy1-18.png', 'happy2-16.png', 'happy1-12.png', 'happy1-14.png', 'happy1-03.png', 'happy1-05.png', 'happy2-11.png']
['sad1-16.png', 'sad2-02.png', 'sad2-15.png', 'sad1-08.png', 'sad1-18.png', 'sad1-15.png', 'sad1-06.png', 'sad1-00.png', 'sad2-10.png', 'sad2-07.png']


In [6]:

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
       def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.999):
            print("\nДостигнута точность 99.9%, обучение прерывается!")
            self.model.stop_training = True


callbacks = myCallback()

# Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])


  # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
  # и train_generator, вызовом train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

# Используйте target_size  150 X 150.
train_generator = train_datagen.flow_from_directory(
   '/tmp/h-or-s/',
        target_size=(150, 150),
        batch_size=10,
        class_mode='binary')
# ожидаемая строка вывода: 'Found 80 images belonging to 2 classes'

# model fitting
history = model.fit(
      train_generator,
      steps_per_epoch=8,
      epochs=20,
      verbose=1,
      callbacks=[callbacks])
# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"


Found 80 images belonging to 2 classes.
Epoch 1/20
8/8 [==============================] - 3s 284ms/step - loss: 3.0207 - accuracy: 0.5750
Epoch 2/20
8/8 [==============================] - 2s 279ms/step - loss: 0.5239 - accuracy: 0.7250
Epoch 3/20
8/8 [==============================] - 3s 433ms/step - loss: 0.3739 - accuracy: 0.8250
Epoch 4/20
8/8 [==============================] - 2s 285ms/step - loss: 0.1876 - accuracy: 0.9250
Epoch 5/20
8/8 [==============================] - 2s 282ms/step - loss: 0.1818 - accuracy: 0.9125
Epoch 6/20
8/8 [==============================] - 2s 278ms/step - loss: 0.1253 - accuracy: 0.9750
Epoch 7/20
8/8 [==============================] - 2s 286ms/step - loss: 0.0656 - accuracy: 0.9750
Epoch 8/20
8/8 [==============================] - 3s 433ms/step - loss: 0.1003 - accuracy: 0.9625
Epoch 9/20
8/8 [==============================] - 2s 278ms/step - loss: 0.0276 - accuracy: 0.9875
Epoch 10/20
8/8 [==============================] - 2s 288ms/step - loss: 0.037